#### Script to create the metadata.jsonl file for donut training

In [1]:
import json
import jsonlines
import pandas as pd

In [2]:
postqc = pd.read_csv('/home/balajic/workdir/donut_training_pipeline/donut_finetuning/database/postqc_dump.csv') # path to postqc file
# postqc_qc_pass = pd.read_csv('/home/balajic/workdir/donut_training_pipeline/database/qc_pass_postqc_dump.csv') # path to postqc file
# postqc_ext_fin = pd.read_csv('/home/balajic/workdir/donut_training_pipeline/database/extraction_finished_postqc_dump.csv') # path to postqc file

# concatenate both the postqc into one biug file
# postqc = pd.concat([postqc_qc_pass, postqc_ext_fin])

In [7]:
postqc.columns

Index(['claim_id', 'current_state', 'claim_amount', 'txn_id', 'id', 'page_id',
       'line_id', 'particular', 'unit_price', 'quantity', 'bd_amount',
       'discount', 'ad_amount', 'l1', 'l2', 'nme', 'request_id', 'table_type',
       'after_discount_amount_score', 'before_discount_amount_score',
       'discount_score', 'particular_score', 'quantity_score', 'row_score',
       'unit_price_score', 'sequence_id', 'bl_x', 'bl_y', 'br_x', 'br_y',
       'tabulated_box', 'tl_x', 'tl_y', 'tr_x', 'tr_y', 'subtotal_hex_id'],
      dtype='object')

In [8]:
postqc[postqc['claim_id']==22620558][['particular', 'bd_amount']]

,particular,bd_amount
28440,day care- room rent,3000.0
28821,surgeon fee,12000.0
28828,iol charges,95000.0
28829,anaesthesia charges,3000.0
28830,ot charges,2000.0


In [3]:
# postqc.columns
# ignore pandas SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
import os
import json
import jsonlines
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
# import pytesseract

# root path to the dataset contains the train, test and validation folders
dataset_path = '/home/balajic/workdir/donut_training_pipeline/multipager_donut1/'
interim_df_folder = '/home/balajic/workdir/donut_training_pipeline/donut_finetuning/interim_df/'
save_path = '/home/balajic/workdir/donut_training_pipeline/'

def process_image(image):
    page_flag = False
    
    try:
        # images are saved in the format of <claim_id>-<txn_id>-page_<page_id>.jpeg
        claim = image.split('-')[0]
        imgname = image.split('-')[-1]
        txn_id = image.replace(f'-{imgname}','').replace(f'{claim}-','')
        img_path = os.path.join(dataset_path, splitt, image)
        img = Image.open(img_path)
        width, height = img.size
        
        # read the intermediate df of bill_extractor
        interim_df_path = os.path.join(interim_df_folder, image + '.csv')
        if os.path.exists(interim_df_path):
            df = pd.read_csv(interim_df_path)

            df.replace("VNA",np.nan,inplace=True)
            df.dropna(axis=1, how='all', inplace=True)
            df.drop(columns=['bb_rowwise', 'con_rowwise'], inplace=True)
            df.fillna("", inplace=True)

            # renaming the columns into standard format
            df.rename(columns={
                "Particular": "particular",
                'Unit_price': 'unit_price',
                'Quantity': 'quantity',
                'After_discount_amount': 'ad_amount',
                'Before_discount_amount': 'bd_amount',
                'Discount': 'discount'
                }, inplace=True)

            pageid = imgname.split('_')[1].split('.')[0]
            # get the request wise postqc data
            req_wise = postqc[postqc['txn_id']==txn_id]
            page_wise = req_wise[req_wise['page_id']==int(pageid)]
            page_wise = page_wise.sort_values(by='line_id')
            max_lines = page_wise.shape[0]
            new_line_id = range(1, max_lines + 1)
            page_wise['line_id'] = new_line_id 
            page_wise.rename(columns={
                'line_id': 'line_id',
                "particular": "particular",
                'unit_price': 'unit_price',
                'quantity': 'quantity',
                'bd_amount': 'bd_amount',
                'discount': 'discount',
                'ad_amount': 'ad_amount'
                }, inplace=True)
            try:
                table_type = page_wise['table_type'].values[0]
            except:
                table_type = 'Detail'

            if len(page_wise) > 0:
                common_column = list(page_wise.columns.intersection(df.columns))
                common_column.append('line_id')
                common_column.append('tl_x')
                common_column.append('tl_y')
                common_column.append('br_x')
                common_column.append('br_y')
                # common_column.append('table_type')
                page_wisee = page_wise[common_column]
                page_wisee.rename(columns={
                    'tl_x': 'tl_x',
                    'tl_y': 'tl_y',
                    'br_x': 'br_x',
                    'br_y': 'br_y'
                    }, inplace=True)
                page_wisee['tl_x'] = page_wisee['tl_x'].astype(float).round().astype(int)
                page_wisee['tl_y'] = page_wisee['tl_y'].astype(float).round().astype(int)
                page_wisee['br_x'] = page_wisee['br_x'].astype(float).round().astype(int)
                page_wisee['br_y'] = page_wisee['br_y'].astype(float).round().astype(int)

                colus = ['particular', 'unit_price', 'quantity', 'bd_amount', 'discount', 'ad_amount', 'limne_id']
                for col in colus:
                    if col in page_wisee.columns:
                        if col == 'particular':
                            page_wisee[col] = page_wisee[col].astype(str)
                        elif col == 'line_id':
                            page_wisee[col] = page_wisee[col].astype(float).round().astype(int)
                        else:
                            page_wisee[col] = page_wisee[col].astype(float)

                # if (df.shape[0] != page_wisee.shape[0]) and (page_wise.current_state.values[0] == 'Extraction_Finished'):
                #     return None

            else:
                page_flag = True
                return None  # return None if no page-wise data

        else:
            page_flag = True
            return None  # return None if no interim data

        if page_flag:
            return {
                "file_name": f"{image}",
                "ground_truth": json.dumps({
                    'gt_parse': {'table': [], 'page_type': 'others', 'table_type': 'others'},
                    'meta': {
                        'version': '2.0.0',
                        'split': f"{splitt}",
                        'image_id': image,
                        'image_size': {'width': width, 'height': height}
                        },
                    'valid_line': [],
                    'roi': [],
                    'repeating_symbol': [],
                    'dontcare': []
                })
            }
        else:
            return {
                "file_name": f"{image}",
                "ground_truth": json.dumps({
                    'gt_parse': {'table': page_wisee.to_dict('records'), 'page_type': 'BILL', 'table_type': table_type},
                    'meta': {
                        'version': '2.0.0',
                        'split': f"{splitt}",
                        'image_id': image,
                        'image_size': {'width': width, 'height': height}
                        },
                    'valid_line': [],
                    'roi': [],
                    'repeating_symbol': [],
                    'dontcare': []
                })
            }
    except Exception as e:
        print(e, image)
        return None

if __name__ == "__main__":
    max_threads = 12
    
    for splitt in ['validation']:
        metadata_accumulator = []
        image_folder = os.path.join(dataset_path, splitt)
        image_files = [f for f in os.listdir(image_folder) if f != 'metadata.jsonl']

        with ProcessPoolExecutor(max_workers=max_threads) as executor:
            results = list(tqdm(executor.map(process_image, image_files), total=len(image_files)))
        
        with jsonlines.open(save_path + f"{splitt}_metadata.jsonl", 'a') as wr:
            for result in results:
                if result:
                    wr.write(result)

        print(f"{splitt.capitalize()} images processed.")


100%|██████████| 987/987 [00:01<00:00, 631.99it/s]

Validation images processed.


In [18]:
import os
images = os.listdir('/home/balajic/workdir/donut_training_pipeline/multipager_donut_original/validation')
with open('//home/balajic/workdir/donut_training_pipeline/train.txt', 'w') as fp:
    fp.write(str(images))

In [19]:
len(images)

5201